In [5]:
import cv2
print(cv2.__version__)

4.10.0


In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #for data augumentetion
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog #sprovides a dialog box to select files.
from PIL import Image, ImageTk #Image Converts leages for display in Thinter GUES
import numpy as np

In [13]:
train_dir = r'C:\Users\kgadg\OneDrive\Desktop\Edunet_SIT\ML\Forest_Fire\train'
test_dir = r'C:\Users\kgadg\OneDrive\Desktop\Edunet_SIT\ML\Forest_Fire\test'
valid_dir = r'C:\Users\kgadg\OneDrive\Desktop\Edunet_SIT\ML\Forest_Fire\valid'

In [17]:
#set up for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

#load images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)


Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


#set up for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

#load images
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='binary'
)


In [18]:
#building a simple CNN model 
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') #binary classification: wildfire or no wildfire
])

#compile the model 
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

# Step 4:Training The model
Train the model with train and valid data

In [20]:
# Train the model
history=model.fit(train_generator,validation_data=valid_generator,epochs=5,verbose=1)

Epoch 1/5


C:\Users\kgadg\anaconda3\envs\Deep_Learning\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


946/946 ━━━━━━━━━━━━━━━━━━━━ 72s 74ms/step - accuracy: 0.8771 - loss: 0.2916 - val_accuracy: 0.9400 - val_loss: 0.1583
Epoch 2/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 68s 72ms/step - accuracy: 0.9301 - loss: 0.1901 - val_accuracy: 0.9490 - val_loss: 0.1402
Epoch 3/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 67s 70ms/step - accuracy: 0.9415 - loss: 0.1652 - val_accuracy: 0.9514 - val_loss: 0.1315
Epoch 4/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 67s 71ms/step - accuracy: 0.9449 - loss: 0.1512 - val_accuracy: 0.9586 - val_loss: 0.1147
Epoch 5/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 67s 71ms/step - accuracy: 0.9516 - loss: 0.1299 - val_accuracy: 0.9608 - val_loss: 0.1098


In [21]:
model.save("ffd_model.keras")

In [24]:
model=load_model("ffd_model.keras")
print("Model loaded successfully")

Model loaded successfully


# Step 5: GUI

In [23]:
from tensorflow.keras.models import load_model

In [ ]:
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)    #convert image for tk
        image_label.configure(image=img) #update the image in GUI
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0] #extracts the scalar prediction value
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image) #button triggers the predict_image() function when clicked
btn.pack(pady=20)

#Placeholder for displaying the selected image
image_label = tk.Label(root)
image_label.pack()

#Label to display the prediction result
result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

#Starts the Tkinter event loop, keeping the GUI active until manually closed
root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
